In [11]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import glob
import ipywidgets
from scipy import constants as C
import IPython.display

base_diagnostic = 'out/test'
uppe_list = glob.glob(base_diagnostic+'*_uppe_wave*.npy')
uppe_list += glob.glob(base_diagnostic+'*_uppe_source*.npy')
uppe_list += glob.glob(base_diagnostic+'*_uppe_plasma*.npy')
para_list = glob.glob(base_diagnostic+'*_paraxial_wave*.npy')
total_list = uppe_list + para_list

mpl.rcParams['text.usetex'] = False
mpl.rcParams['font.size'] = 18
l1_mm = 1e3*0.8e-6/(2*np.pi)
t1_ps = 1e12*0.8e-6/(2*np.pi)/C.c

file_w = ipywidgets.Dropdown(options=total_list,value=total_list[0],description='File')
frame_w = ipywidgets.IntSlider(min=0,max=0,step=1,value=0,continuous_update=False)
disp_w = ipywidgets.Dropdown(options=['Falsecolor2D','Lineout'],value='Falsecolor2D',description='Display')
rep_w = ipywidgets.Dropdown(options=['Time','Fourier'],value='Time',description='Representation')
color_w = ipywidgets.Dropdown(options=['viridis','jet','plasma','inferno','ocean','seismic','bwr','prism',
                                      'nipy_spectral'],value='viridis',description='Color')
shape_w = ipywidgets.Textarea(value='no data',description='Shape')
bounds_w = ipywidgets.Textarea(value='no data',description='Bounds')

def load_data(file):
    global A,real_field,data_ext
    real_field = file in uppe_list
    A = np.load(file)
    s = file.split('_')
    data_ext_name = s[0] + '_' + s[1] + '_' + s[2] + '_plot_ext.npy'
    data_ext = np.load(data_ext_name)
    data_ext = np.concatenate((data_ext[0:2],l1_mm*data_ext[2:4]))
    bounds_w.value = str(data_ext)
    shape_w.value = str(A.shape)
    frame_w.value = 0
    frame_w.max = A.shape[3]-1

def load(file):
    load_data(file)
    change_rep_data(rep_w.value)
    disp_frame(file,frame_w.value,disp_w.value,rep_w.value,color_w.value)

def get_freq_time_data(A,ext):
    if real_field:
        # Nodes are like [0,1,2,3] and walls are like [-0.5,0.5,1.5,2.5,3.5]
        # In these examples the user's requested upper bound would be 4 (it is thrown out)
        # Therefore element N/2+1 should be regarded as the central frequency
        dw = (ext[1] - ext[0])/A.shape[0]
        wc = 0.5*ext[0] + 0.5*(ext[1] + dw)
        tmax = 2*np.pi/dw
        return dw,wc,tmax
    else:
        # Nodes are like [-2,-1,0,1] and walls are like [-2.5,-1.5,-.5,.5,1.5]
        dw = (ext[1] - ext[0])/A.shape[0]
        wc = 0.5*ext[0] + 0.5*(ext[1] + dw)
        tmax = 2*np.pi/dw
        return dw,wc,tmax

def change_rep_data(rep):
    global plot_data,hstr,vstr,cstr,plot_ext
    plot_ext = np.copy(data_ext)
    if rep=='Time' and real_field:
        dw,wc,tmax = get_freq_time_data(A,plot_ext)
        plot_ext[0] = 0.0
        plot_ext[1] = t1_ps*tmax
        hstr = r'$t-z/c$ (ps)'
        vstr = r'$\rho$ (mm)'
        cstr = r'$eA(t)/mc^2$'
        passband = (0,8.0)
        w_array = np.linspace(data_ext[0],data_ext[1],A.shape[0])[...,np.newaxis,np.newaxis,np.newaxis]
        filter_function = lambda w : 0.5*(1+np.tanh((passband[1]-w)/.01)) * 0.5*(1+np.tanh((w-passband[0])/.01))
        filter_vec = filter_function(w_array)
        plot_data = np.fft.irfft(np.conj(A)*filter_vec,axis=0)
    if rep=='Time' and not real_field:
        dw,wc,tmax = get_freq_time_data(A,plot_ext)
        plot_ext[0] = 0.0
        plot_ext[1] = t1_ps*tmax
        hstr = r'$t-z/c$ (ps)'
        vstr = r'$x$ (mm)'
        cstr = r'$|a|(t)$'
        plot_data = np.abs(np.fft.ifft(np.fft.ifftshift(np.conj(A),axes=0),axis=0))
    if rep=='Fourier':
        hstr = r'$\omega/\omega_0$'
        vstr = r'$x$ (mm)'
        cstr = r'$|a|(\omega)$'
        plot_data = np.abs(A)
        #plot_data = np.log10(plot_data+1e-20)

def change_rep(rep):
    change_rep_data(rep)
    disp_frame(file_w.value,frame_w.value,disp_w.value,rep,color_w.value)

def disp_frame(file,frame,disp,rep,color):
    tslice = np.int(A.shape[0]/2)
    yslice = np.int(A.shape[2]/2)
    plt.figure(1,figsize=(16,7))
    if disp=='Lineout':
        x = np.linspace(plot_ext[0],plot_ext[1],plot_data.shape[0])
        plt.plot(x,plot_data[:,yslice,yslice,frame])
        #x = np.linspace(plot_ext[2],plot_ext[3],plot_data.shape[1])
        #plt.plot(x,plot_data[tslice+4,:,yslice,frame])
    else:
        #plt.imshow(plot_data[:,:,yslice,frame].swapaxes(0,1),vmin=-.0002,vmax=.0002,origin='lower',cmap=color,aspect=0.7,extent=plot_ext)
        plt.imshow(plot_data[:,:,yslice,frame].swapaxes(0,1),vmin=np.min(plot_data),vmax=np.max(plot_data),origin='lower',cmap=color,aspect='auto',extent=plot_ext)
        #plt.imshow(plot_data[:,:,yslice,frame].swapaxes(0,1),origin='lower',cmap=color,aspect='auto',extent=plot_ext)
        b=plt.colorbar()
        b.set_label(cstr,size=24)
    #plt.xlim(0.25,0.45)
    plt.xlabel(hstr,size=24)
    plt.ylabel(vstr,size=24)
    plt.tight_layout()
    plt.show()

rep_iw = ipywidgets.interactive(change_rep,rep=rep_w)
load_iw = ipywidgets.interactive(load,file=file_w)
update_iw = ipywidgets.interactive(disp_frame,file=file_w,frame=frame_w,disp=disp_w,rep=rep_w,color=color_w)

left_box = ipywidgets.VBox([file_w,frame_w,disp_w,rep_w,color_w,shape_w,bounds_w],
                            layout=ipywidgets.Layout(justify_content='flex-start',width='800px',flex='1 3 auto'))

update_iw.children[-1].layout.flex = '4 2 auto'
main_view = ipywidgets.HBox([left_box, update_iw.children[-1]],
                            layout=ipywidgets.Layout(display='inline-flex',
                                                     align_items='stretch',
                                                     align_content='stretch',
                                                     justify_content='flex-start'))

load_data(file_w.value)
change_rep_data('Time')
IPython.display.display(main_view)